# Help Desk Scheduling with PuLP (Colab Ready)

This notebook shows how to experiment with the standalone `scheduler_lp` module in a cloud environment such as Google Colab.
It downloads the latest module files directly from GitHub, installs the required solver dependency, and
walks through building a toy rostering instance that you can adapt for your own data.

## 1. Environment setup
Run the next two cells once per session to install dependencies and fetch the module files.
Feel free to replace the GitHub branch or file URLs if you want to test local changes.

In [ ]:
# Install dependencies needed for scheduling and Excel ingestion.
%pip install --quiet pulp==2.7.0 pandas openpyxl

In [ ]:
"""Download the scheduler_lp module from GitHub so we can import it locally."""
from pathlib import Path
from urllib.request import urlopen

RAW_BASE = "https://raw.githubusercontent.com/firepenguindisopanda/INFO3604-help-desk-rostering/routes_v2_fix/scheduler_lp"
FILES = {
    "__init__.py": f"{RAW_BASE}/__init__.py",
    "linear_scheduler.py": f"{RAW_BASE}/linear_scheduler.py",
    "examples.py": f"{RAW_BASE}/examples.py",
}

target_dir = Path("scheduler_lp")
target_dir.mkdir(parents=True, exist_ok=True)

for name, url in FILES.items():
    with urlopen(url) as response:
        data = response.read()
    (target_dir / name).write_bytes(data)

print(f"Downloaded {len(FILES)} files into {target_dir.resolve()}")

## 2. Load exported schedule data
Use the `scripts/export_scheduler_data.py` helper to generate an Excel workbook locally (or on your VM) and upload it here. The default name in this notebook is `helpdesk_scheduler_inputs.xlsx`, but you can change it if needed.

In [ ]:
from datetime import datetime
from pathlib import Path

import pandas as pd

EXCEL_PATH = Path("helpdesk_scheduler_inputs.xlsx")
if not EXCEL_PATH.exists():
    raise FileNotFoundError(
        f"Expected to find {EXCEL_PATH} in the working directory. Upload the Excel export or update EXCEL_PATH."
    )

assistants_df = pd.read_excel(EXCEL_PATH, sheet_name="assistants")
availability_df = pd.read_excel(EXCEL_PATH, sheet_name="assistant_availability")
courses_df = pd.read_excel(EXCEL_PATH, sheet_name="assistant_courses")
shifts_df = pd.read_excel(EXCEL_PATH, sheet_name="shifts")
shift_demands_df = pd.read_excel(EXCEL_PATH, sheet_name="shift_course_demands")

print("Loaded sheets:")
for name, df in {
    "assistants": assistants_df,
    "assistant_availability": availability_df,
    "assistant_courses": courses_df,
    "shifts": shifts_df,
    "shift_course_demands": shift_demands_df,
}.items():
    print(f"  - {name}: {len(df)} rows")

## 3. Build rostering objects from the Excel data
With the sheets in memory we can convert each row into the dataclasses consumed by `scheduler_lp.linear_scheduler`. Feel free to adapt the transformation logic if your organisation tracks additional metadata.

In [ ]:
from datetime import time

from scheduler_lp import (
    AvailabilityWindow,
    Assistant,
    CourseDemand,
    SchedulerConfig,
    Shift,
    solve_helpdesk_schedule,
)

In [ ]:
def _ensure_time(value):
    if isinstance(value, str):
        return datetime.strptime(value, "%H:%M").time()
    if hasattr(value, "to_pydatetime"):
        return value.to_pydatetime().time()
    if hasattr(value, "hour"):
        return value
    raise ValueError(f"Unrecognised time value: {value!r}")


availability_map = {
    assistant_id: [
        AvailabilityWindow(
            day_of_week=int(row.day_of_week),
            start=_ensure_time(row.start_time),
            end=_ensure_time(row.end_time),
        )
        for row in group.itertuples(index=False)
    ]
    for assistant_id, group in availability_df.groupby("assistant_id")
}

course_map = {
    assistant_id: sorted({row.course_code.upper() for row in group.itertuples(index=False)})
    for assistant_id, group in courses_df.groupby("assistant_id")
}

assistants = []
for row in assistants_df.itertuples(index=False):
    if not bool(row.active):
        continue
    assistants.append(
        Assistant(
            id=row.assistant_id,
            courses=course_map.get(row.assistant_id, []),
            availability=availability_map.get(row.assistant_id, []),
            min_hours=float(row.hours_minimum or 0),
            max_hours=None,
            cost_per_hour=float(row.rate or 0.0),
        )
    )

shift_demands_grouped = {
    shift_id: [
        CourseDemand(
            course_code=row.course_code.upper(),
            tutors_required=int(row.tutors_required),
            weight=float(row.weight) if pd.notna(row.weight) else float(row.tutors_required),
        )
        for row in group.itertuples(index=False)
    ]
    for shift_id, group in shift_demands_df.groupby("shift_id")
}

shifts = []
for row in shifts_df.itertuples(index=False):
    demands = shift_demands_grouped.get(row.shift_id, [])
    if not demands:
        continue
    min_staff = max(1, int(sum(d.tutors_required for d in demands)))
    shifts.append(
        Shift(
            id=str(row.shift_id),
            day_of_week=int(row.day_of_week),
            start=_ensure_time(row.start_time),
            end=_ensure_time(row.end_time),
            course_demands=demands,
            min_staff=min_staff,
            metadata={
                "schedule_id": str(row.schedule_id),
                "date": str(row.date),
            },
        )
    )

print(f"Active assistants loaded: {len(assistants)}")
print(f"Shifts with demand: {len(shifts)}")

In [ ]:
config = SchedulerConfig(
    course_shortfall_penalty=5.0,
    understaffed_penalty=40.0,
    min_hours_penalty=12.0,
    max_hours_penalty=5.0,
    solver_time_limit=120,
    log_solver_output=False,
)

if not assistants or not shifts:
    raise ValueError("No assistants or shifts were generated. Check the Excel input sheets.")

result = solve_helpdesk_schedule(assistants, shifts, config=config)

print(f"Solver status: {result.status}")
if result.objective_value is not None:
    print(f"Objective value: {result.objective_value:.3f}")

print("\nAssignments:")
if result.assignments:
    for assistant_id, shift_id in result.assignments:
        print(f"  - {assistant_id} -> {shift_id}")
else:
    print("  (none)")

print("\nHours by assistant:")
for assistant_id, hours in sorted(result.assistant_hours.items()):
    print(f"  - {assistant_id}: {hours:.2f} hours")

print("\nCourse shortfalls (shift_id, course_code):")
for key, value in sorted(result.course_shortfalls.items()):
    print(f"  - {key}: {value:.2f}")

print("\nStaff shortfalls (shift_id -> understaffed amount):")
for shift_id, value in sorted(result.staff_shortfalls.items()):
    print(f"  - {shift_id}: {value:.2f}")

## 4. Next steps
- Tweak the transformation logic (e.g., cap weekly hours) to mirror your policies.
- Adjust penalty weights in `SchedulerConfig` to reflect your priorities.
- Inspect `result.to_assignment_matrix()` for easier downstream formatting.
- Use the downloaded `examples.py` for more ideas ("Run Module" > `python -m scheduler_lp.examples`).